In [ ]:
import os
import argparse
import kubeflow.katib as kc
import time
import logging
from datetime import datetime, timezone
from kubernetes.client import V1ObjectMeta

class CannotRunError(Exception):
  """Base class for exceptions in this module."""
  def __init__(self, message):
      self.message = message

class MonitorKatibJob(object):
    
  def getLocalTime(self):
    return datetime.now(timezone.utc).astimezone().isoformat()

  def printNowStatus(self, experiment_name, user_namespace):
    self.now_status = kc.KatibClient().get_experiment_status(name=experiment_name, namespace=user_namespace)
    if self.now_status != self.prev_status:
      print(f"now status : {self.now_status}", flush=True)    
    self.prev_status = self.now_status
    time.sleep(1)      
       
  def printOptimalHp(self, experiment_name, user_namespace):      
    optimal_hp = kclient.get_optimal_hyperparmeters(name=experiment_name, namespace=user_namespace)
    print(f" optimal_hp = {optimal_hp}", flush=True) 

  def monitor(self):
    parser= argparse.ArgumentParser()
    parser.add_argument("--experiment_name", required=True, type=str)
    parser.add_argument("--namespace", required=False, type=str)                
    parser.add_argument("--trial_name", required=False, type=str)        
    args = parser.parse_args()

    user_namespace = args.namespace
    experiment_name =  args.experiment_name

    kclient = kc.KatibClient()

    # waiting 2min
    waiting_running_count = 120
    now_count = 0
    self.prev_status = "prev"
    self.now_status = ""
    
    print(f"Start Monitoring {experiment_name}", flush=True)
    
    # check Stated to Running
    for now_time in range(waiting_running_count):
      self.printNowStatus(experiment_name, user_namespace)  
        
      if self.now_status == "Succeeded":
        printOptimalHp(experiment_name, user_namespace)
        break;
        
      # last Phase  
      if self.now_status == "Running":
        print(f"{experiment_name} is Running!", flush=True)
        break

    if self.now_status != "Running" and self.now_status != "Succeeded":
      raise CannotRunError('Not Gooood!')
    
    # monitor experiment succeed

    while self.now_status == "Running":
      experiment_result = kclient.is_experiment_succeeded(name=experiment_name, namespace=user_namespace)
      optimal_hp = kclient.get_optimal_hyperparmeters(name=experiment_name, namespace=user_namespace)
      optimal_trial_exist = optimal_hp["currentOptimalTrial"]["observation"]["metrics"]
      if optimal_trial_exist:
        current_best = optimal_hp["currentOptimalTrial"]["observation"]["metrics"][0]["value"]    
        print(f"Now Best : {current_best}", flush=True)
        
      self.printNowStatus(experiment_name, user_namespace)  

      if experiment_result:
        print(f" optimal_hp = {optimal_hp}", flush=True)
        break
        
        # findMax
        
if __name__=="__main__":
    if os.getenv('FAIRING_RUNTIME', None) is None:
        """
        from kubeflow import fairing
        from kubeflow.fairing.kubernetes import utils as k8s_utils

        DOCKER_REGISTRY = 'kubeflow-registry.default.svc.cluster.local:30000'
        fairing.config.set_builder(
            'append',
            image_name='monitor-katib-job',
            base_image='brightfly/katib-sdk:0.0.1',
            registry=DOCKER_REGISTRY,
            push=True)
        # cpu 1, memory 1GiB
        fairing.config.set_deployer('job',
                                    namespace='amaramusic'
                                    )
        fairing.config.run()    
        """
        from kubeflow.fairing.builders.append.append import AppendBuilder
        from kubeflow.fairing.preprocessors.converted_notebook import ConvertNotebookPreprocessor

        DOCKER_REGISTRY = 'kubeflow-registry.default.svc.cluster.local:30000'
        base_image='brightfly/katib-sdk:0.0.1'
        image_name='monitor-katib-job'

        builder = AppendBuilder(
            registry=DOCKER_REGISTRY,
            image_name=image_name,
            base_image=base_image,
            push=True,
            
            preprocessor=ConvertNotebookPreprocessor(
                notebook_file="monitor_for_katib.ipynb" )
            )
        builder.build()        
    else:
        job = MonitorKatibJob()
        job.monitor()

In [ ]:
!pip install kubeflow-katib==0.0.2